In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Another option to get the table
```python
import wikipedia as wp
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8") #Get the html source
df = pd.read_html(html)[0]       # get the table to df
df.to_csv('beautifulsoup_pandas.csv',header=0,index=False) # save the table to csv
new_header = df.iloc[0]          #grab the first row for the header
df = df[1:]                      #take the data less the header row
df.columns = new_header          #set the header row as the df header
df.head()
```

Read the source code for a given web page and creating a BeautifulSoup (soup)object 

In [2]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())    # print the full text 

Find class ‘wikitable sortable’ in the HTML script

In [3]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

Each row is start with **tr** and end with **/tr**. Each cell is start with **td** and end with **/td**.
Get the header of the table

In [4]:
ths = table.find_all('th')
headings = [th.text.strip() for th in ths]
headings

['Postcode', 'Borough', 'Neighbourhood']

Get each row and append to a list

In [5]:
lists=[]
for row in table.findAll("tr"):
    cells = row.findAll("td")
    items = [th.text.strip() for th in cells]
    lists.append(items)

Convert the list to dataframe

In [6]:
df=pd.DataFrame(lists[1:])
df.columns=headings
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop rows if it contains a 'Not assigned'in Borough column.

In [7]:
df = df[df.Borough != 'Not assigned']

Merge rows with same Postcode and Borough

In [8]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x:','.join(x.astype(str))).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Replace Not assigned neighborhood with its borough.

In [9]:
Vpostcode = df['Borough']
df['Neighbourhood']=Vpostcode.where(df['Neighbourhood']=='Not assigned',other=df['Neighbourhood'])
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df.shape

(103, 3)

In [11]:
path = 'http://cocl.us/Geospatial_data'
gosdf = pd.read_csv(path)
gosdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
neighborhoods = pd.concat([df, gosdf], axis=1, join='inner')
neighborhoods = neighborhoods.drop('Postal Code',axis=1)
neighborhoods.rename(columns={'Postcode': 'PostalCode'}, inplace=True)
neighborhoods.to_csv('toronto.csv',index=False,header=True)    # write data to file
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
